Lite saker att tänka på
- starta om kernel mellan körningar, man verkar behöva cleara simulation variabeln som skapas
- radera md_log.txt och output.txt mellan körningar, koden appendar bara och skapar inte nya filer om man kör cellen fler ggr

In [1]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout

#pdb = PDBFile("F154V_F174V_serin.pdbqt")
#pdb = PDBFile("2q0s_serine_clean.pdb")
#pdb = PDBFile("F149V_F153V_serine.pdb")
pdb = PDBFile("2q0s_pdb1.pdb")

In [2]:

# Specify the forcefield
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')

In [3]:
modeller = Modeller(pdb.topology, pdb.positions)
modeller.deleteWater()


In [4]:
residues=modeller.addHydrogens(forcefield)

ValueError: No template found for residue 10 (SER).  The set of atoms matches NGLU, but the bonds are different.  For more information, see https://github.com/openmm/openmm/wiki/Frequently-Asked-Questions#template

## system setup

In [5]:
#system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME, nonbondedCutoff=1.5*nanometer, constraints=HBonds)
system = forcefield.createSystem(modeller.topology, nonbondedMethod=NoCutoff, constraints=HBonds)
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.004*picoseconds)
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)

## inledande minimering

In [6]:
print("Minimizing energy")
simulation.minimizeEnergy()



Minimizing energy


## setup simulation

In [7]:
simulation.reporters.append(PDBReporter('output.pdb', 1000))
simulation.reporters.append(StateDataReporter(stdout, 100, step=True,
        potentialEnergy=True, temperature=True, volume=True))
simulation.reporters.append(StateDataReporter("md_log.txt", 100, step=True,
        potentialEnergy=True, temperature=True, volume=True))

## starta simulation

In [8]:
print("Running NVT")
simulation.step(10000)

Running NVT
#"Step","Potential Energy (kJ/mole)","Temperature (K)","Box Volume (nm^3)"
100,-84241.08135986328,100.6008100762887,8.0
200,-66282.71923828125,168.674696804876,8.0
300,-54891.87567138672,213.74233792712783,8.0
400,-46843.94323730469,244.04387860708712,8.0
500,-42049.73352050781,262.9192765120905,8.0
600,-39291.85876464844,277.1482914350782,8.0
700,-36264.33996582031,283.56658194008577,8.0
800,-34478.40710449219,288.5337619244022,8.0
900,-33995.791015625,295.8511041080548,8.0
1000,-32583.555786132812,296.3726733765304,8.0
1100,-32569.57421875,298.35911648273594,8.0
1200,-31875.33154296875,296.9110805126113,8.0
1300,-33353.227294921875,301.47507162596816,8.0
1400,-33557.883728027344,298.9399084640825,8.0
1500,-33585.26306152344,297.2261340560412,8.0
1600,-32832.85498046875,299.1872326524426,8.0
1700,-32954.80615234375,300.9451713202625,8.0
1800,-33087.6884765625,301.6520954789006,8.0
1900,-33287.1181640625,302.2241933961618,8.0
2000,-33046.89208984375,301.13952459508664,8.0
2

## minimera efter dynamik

In [9]:
print("Minimizing energy")
simulation.minimizeEnergy()

Minimizing energy


## Energi plottat mot step

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
data = np.loadtxt("md_log.txt", delimiter=',')

step = data[:,0]
potential_energy = data[:,1]
temperature = data[:,2]
volume = data[:,3]

plt.plot(step, potential_energy)
plt.xlabel("Step")
plt.ylabel("Potential energy (kJ/mol)")
plt.show()
plt.plot(step, temperature)
plt.xlabel("Step")
plt.ylabel("Temperature (K)")
plt.show()
plt.plot(step, volume)
plt.xlabel("Step")
plt.ylabel("Volume (nm^3)")
plt.show()


## sparar resultat från simulation

In [10]:
positions = simulation.context.getState(getPositions=True).getPositions()
PDBFile.writeFile(simulation.topology, positions, open('F153V_F173V_serine_md.pdb', 'w'))

## visulalisering


In [ ]:
import nglview as nv
import MDAnalysis as mda

In [ ]:
u = mda.Universe("output.pdb", "output.pdb")

all = u.select_atoms('all')
w = nv.show_mdanalysis(all)
w.add_representation('licorice',selection="10")

w

In [ ]:
def show_residues_around(view, component_index=0, selection="10", radius=5.0):
    js = (
        f"""
        // Get first (and only) loaded component: our protein-ligand system
        var system = this.stage.compList[{component_index}];
        // Store current selection, we will need it laer
        var prevSele = system.selection.string;
        // Set selection to our desired ligand
        system.setSelection("{selection}");
        // Select all atoms within 5A from the ligand
        var around = system.structure.getAtomSetWithinSelection(system.selection, {radius});
        """
        """
        // Extend selection so it includes full residues
        var around_complete = system.structure.getAtomSetWithinGroup(around);
        // Add representation for those atoms
        system.addRepresentation("licorice", {sele: around_complete.toSeleString()});
        // Restore selection to original one; otherwise changes won't be reflected
        system.setSelection(prevSele)
        """
    )
    view._execute_js_code(js)

In [ ]:
view = nv.show_mdanalysis(all)
show_residues_around(view)
view.add_representation('ball-and-stick',selection="10")
view.center(selection="10")
view